## Text to Image Generation MonsterAPI Workflow:

MonsterAPI offers you an easy to use and scalable API for using Stable Diffusion v1.5 AI Model to generate high fidelity images from your text instructions (prompts).

MonsterAPI is 50% low cost than other alternatives because it relies on Decentralised GPU computing (powered by Q Blocks).

Enter your Monster X API KEY and BEARER TOKEN below.

If you don't have them then do the following:

1.   Sign up/login on [MonsterAPI](https://monsterapi.ai/signup)
2.   Go to Auth Keys page [here](https://monsterapi.ai/user/settings#api-keys).
3.   Click on "Generate new Key" and enter any random name. Click "Generate".
4.   Once generated, copy the "X API KEY" and "BEARER TOKEN" values.
5.   Paste both values below:



In [ ]:
MONSTER_X_API_KEY = '';
BEARER_AUTH_TOKEN = '';

### In this section you can specify the parameters for image generation:

In [ ]:
prompt          = "b&w photo of 42 y.o man in black clothes, bald, face, half body, body, high detailed skin, skin pores, coastline, overcast weather, wind, waves, 8k uhd, dslr, soft lighting, high quality, film grain, Fujifilm XT3";
negative_prompt = "(semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck";
num_images      = 2;                #int: 1 to 4 
sampling_steps  = 50;               #int: 30 to 500
aspect_ratio    = "portrait";       #string: square, landscape, portrait
guidance_scale  = 10;               #float: 5.0 to 15.0
seed            = 101837751;        #random integer value

Now we define the functions for sending request to MonsterAPI for image generation and fetching the status of our request:

In [ ]:
import requests
import json
import time

# Function for sending request for image generation
def send_request():
  url = "https://api.monsterapi.ai/apis/add-task"
  payload = json.dumps({
    "model": "txt2img",
    "data": {
      "prompt": prompt,
      "negprompt": negative_prompt,
      "samples": num_images,
      "steps": sampling_steps,
      "aspect_ratio": aspect_ratio,
      "guidance_scale": guidance_scale,
      "seed": seed
    }
  })
  headers = {
    'x-api-key': MONSTER_X_API_KEY,
    'Authorization': 'Bearer '+BEARER_AUTH_TOKEN,
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  if response.status_code == 200:
        response_json = response.json()
        process_id = response_json.get('process_id')
        return process_id
  else:
      print(f"Error: {response.status_code}, {response.text}")
      return None

# Function for fetching the status of the image generation request
def fetch_status(process_id):
    url = "https://api.monsterapi.ai/apis/task-status"
    payload = json.dumps({"process_id": process_id})
    headers = {
        'x-api-key': MONSTER_X_API_KEY,
        'Authorization': 'Bearer ' + BEARER_AUTH_TOKEN,
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    if response.status_code == 200:
        response_json = response.json()
        status = response_json.get('response_data', {}).get('status')
        if status == "COMPLETED":
            output = response_json.get('response_data', {}).get('result', {}).get('output', [])
            return status, output
        elif status == "FAILED":
            error = response_json.get('response_data', {}).get('result', {}).get('errorMessage')
            return status, error
        else:
            return status, []
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None, []

Here we finally send the request and display the generated image(s):

In [ ]:
# Send image generation request
process_id  = send_request();

# Fetch status and display generated images on getting "COMPLETED" status:
from IPython.display import Image, display

max_retries = 20
wait_time = 3
retry_count = 0
output = []

if process_id is not None:
  while retry_count < max_retries:
      status, output = fetch_status(process_id)

      if status in ["COMPLETED", "FAILED"]:
          if status == "COMPLETED":
              print("Output:", output)
              for image_url in output:
                  display(Image(url=image_url))
          else:
              print("Task failed\nError: "+output)
          break
      else:
        print("Status: ", status)
      retry_count += 1
      time.sleep(wait_time)

  if retry_count == max_retries:
      print("Max retries exceeded")
else:
  print("Task failed\nUnknown Error")

Status:  IN_QUEUE
Status:  IN_QUEUE
Status:  IN_QUEUE
Status:  IN_QUEUE
Status:  IN_QUEUE
Status:  IN_PROGRESS
Status:  IN_PROGRESS
Status:  IN_PROGRESS
Status:  IN_PROGRESS
Output: ['https://processed-model-result.s3.us-east-2.amazonaws.com/08e795aa-e03f-11ed-96ec-cf845949b6c8_0.png', 'https://processed-model-result.s3.us-east-2.amazonaws.com/08e795aa-e03f-11ed-96ec-cf845949b6c8_1.png']
